In [ ]:
import cv2
import numpy as np
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load the COCO class labels and the pre-trained YOLOv3 model
labels_path = 'yolo/coco.names'
weights_path = 'yolo/yolov3.weights'
config_path = 'yolo/yolov3.cfg'
classes = []
with open(labels_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
net = cv2.dnn.readNet(weights_path, config_path)

# Load the video source
video_capture = cv2.VideoCapture(0)

# Set up the video output window
cv2.namedWindow('Video',cv2.WINDOW_NORMAL)

while True:
    # Read a single frame from the video source
    ret, frame = video_capture.read()
    
    # Create a blob from the input image and pass it through the YOLOv3 model
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(output_layers)
    
    # Loop over all detected objects and draw a bounding box around them
    class_ids = []
    confidences = []
    boxes = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])
                x = center_x - w // 2
                y = center_y - h // 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in indices:
        i = i
        box = boxes[i]
        x, y, w, h = box
        label = f"{classes[class_ids[i]]}: {int(confidences[i]*100)}"
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        engine.say(label)
        engine.runAndWait()

    # Display the resulting image in the video output window
    cv2.imshow('Video', frame)
    
    # Wait for a key press, and exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video source and close the video output window
video_capture.release()
cv2.destroyAllWindows()